## Window Functions

### Rank product based on sales

#### Using `ROW_NUMBER()`

In [ ]:

WITH filtered_sales AS (
    SELECT
        o.product_id,
        o.total_revenue
    FROM 
        orders o
)

SELECT
    P.*,
    ROW_NUMBER() OVER (
        PARTITION BY product_id 
        ORDER BY fs.total_revenue DESC
    ) AS sales_rank
    
FROM 
    filtered_sales fs
JOIN
    products p
USING(product_id);



#### Using `RANK()` and `DENSE_RANK()`

In [ ]:
WITH filtered_sales AS (
    SELECT
        o.product_id,
        o.quantity,
        o.total_revenue
    FROM 
        orders o
)

SELECT
    P.*,
    fs.total_revenue,
    fs.quantity,
    RANK() OVER (
        PARTITION BY product_id 
        ORDER BY 
            fs.total_revenue DESC,
            fs.quantity DESC
    ) AS sales_rank
    
FROM 
    filtered_sales fs
JOIN
    products p
USING(product_id);

#### Using `DENSE_RANK()`

In [ ]:
WITH filtered_sales AS (
    SELECT
        o.product_id,
        o.quantity,
        o.total_revenue
    FROM 
        orders o
)

SELECT
    P.*,
    fs.total_revenue,
    fs.quantity,
    DENSE_RANK() OVER (
        PARTITION BY product_id 
        ORDER BY 
            fs.total_revenue DESC,
            fs.quantity DESC
    ) AS sales_rank
    
FROM 
    filtered_sales fs
JOIN
    products p
USING(product_id);

### Running total sales

In [ ]:

WITH filtered_sales AS (
    SELECT
        o.product_id,
        o.total_revenue
    FROM 
        orders o
)

SELECT
    P.*,
    fs.total_revenue,
    SUM(fs.total_revenue) OVER (
        PARTITION BY 
            p.category 
        ORDER BY 
            fs.total_revenue DESC
    ) AS running_total_sales
    
FROM 
    filtered_sales fs
JOIN
    products p
USING(product_id)


### Average order value for customer using `PARTITION BY`

In [ ]:
-- SELECT 
--     *
-- FROM 
--     orders;


-- WITH customer_avg_order AS (
--     SELECT 
--         customer_id,
--         ROUND(AVG(total_revenue), 2) AS avg_order_value
--     FROM 
--         orders
--     GROUP BY 
--         customer_id
-- )

-- SELECT 
--     c.*,
--     cao.avg_order_value
-- FROM 
--     customers c
-- JOIN 
--     customer_avg_order cao
-- USING(customer_id);

-- Same solution using `PARTITION BY`

SELECT DISTINCT
    C.*,
    ROUND(AVG(o.total_revenue) OVER(
        PARTITION BY c.customer_id
    ), 2) AS avg_order_value
FROM 
    orders o
JOIN
    customers c
USING(customer_id)
ORDER BY 
    c.customer_id;





### Compare monthly sales with `LAG()` and `LEAD()`

In [ ]:
SELECT 
    *,
    LAG(total_revenue, 1, 0) OVER(
        PARTITION BY month
    ) AS previous_month_revenue,
    LEAD(total_revenue, 1, 0) OVER(
        PARTITION BY month
    ) AS next_month_revenue
FROM 
    orders;

### Analyze sales trend and customer behavior

In [ ]:
SELECT 
    product_id,
    category,
    customer_id,
    order_date,
    total_revenue,
    -- Running total of orders for each product category
    SUM(total_revenue) OVER (PARTITION BY category ORDER BY order_date 
                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
    -- Rank of each sale within its category based on total_revenue
    RANK() OVER (PARTITION BY category ORDER BY total_revenue DESC) AS category_rank,
    -- Total orders for each customer
    SUM(total_revenue) OVER (PARTITION BY customer_id) AS total_orders_per_customer,
    -- Average sale total_revenue for each customer
    ROUND(AVG(total_revenue) OVER (PARTITION BY customer_id), 2) AS avg_sale_per_customer,
    -- Number of orders made by each customer
    COUNT(*) OVER (PARTITION BY customer_id) AS orders_count_per_customer,
    -- Difference in orders total_revenue compared to the previous sale for the same customer
    total_revenue - LAG(total_revenue) OVER (PARTITION BY customer_id ORDER BY order_date) AS sales_diff
FROM 
    orders
JOIN 
    products 
USING(product_id)
ORDER BY 
    category, order_date;
